In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import nltk
import plotly.express as px

nltk.download('punkt')

def scrape_capitol_trades():
    headers = {"User-Agent": "Mozilla/5.0"}
    trades = []
    page = 1

    # Loop until no more pages are found
    while True:
        # last 180 days
        url = f"https://www.capitoltrades.com/trades?txDate=180d&page={page}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        #finds the table on the site containing the trades
        table = soup.find('table', {'class': 'trades-table'})
        rows = table.find_all('tr') if table else []

        # If no rows are found, break the loop as there are no more trades
        if not rows:
            break

        for row in rows:
            columns = row.find_all('td')
            if len(columns) > 0:
                trade_data = {
                    'politician': columns[0].text.strip(),
                    'trade': columns[1].text.strip(),
                    'traded': columns[3].text.strip(),
                    'filedafter': columns[4].text.strip(),
                    'owner': columns[5].text.strip(),
                    'type': columns[6].text.strip(),
                    'size': columns[7].text.strip(),
                    'price': columns[8].text.strip(),
                    'link': row.find('a')['href'] if row.find('a') else ''
                }
                trades.append(trade_data)

        # move to the next page
        page += 1

    df = pd.DataFrame(trades)

    # company name and symbol from the 'trade' column using regular expressions
    def extract_company_symbol(trade):
        # capture company name and symbol
        match = re.match(r'^(.*?)([A-Z\.]{1,5})(:US)?$', trade)
        if match:
            return match.group(1).strip(), match.group(2) + (":US" if match.group(3) else "")
        else:
            return trade.strip(), 'N/A'

    df[['company_name', 'symbol']] = df['trade'].apply(lambda x: pd.Series(extract_company_symbol(x)))

    df['price'] = df['price'].str.strip().str.upper()

    # list of symbols to filter by
    symbols = ["AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "BRK.B", "TSLA", "META", "TSM", "V",
               "UNH", "JNJ", "XOM", "WMT", "JPM", "LLY", "PG", "MA", "HD", "CVX", "MRK", "ABBV",
               "PEP", "KO", "AVGO", "PFE", "NVO", "TMO", "ASML", "CSCO", "NFLX", "ORCL", "COST",
               "DIS", "NKE", "MCD", "DHR", "ACN", "LIN", "AMD", "TXN", "VZ", "ABT", "UPS", "NEE",
               "PM", "RTX", "SBUX", "AMGN", "HON"]

    df['symbol_normalized'] = df['symbol'].str.replace(':US', '')
    df_filtered = df[df['symbol_normalized'].isin(symbols)]

    return df_filtered

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
# Scrape the trades data from the last 180 days
capitol_trades_df = scrape_capitol_trades()

# Calculate gains or losses
size_mapping = {
    '1K–15K': 7500,
    '15K–50K': 32500,
    '50K–100K': 75000,
    '100K–250K': 175000,
    '250K–500K': 375000,
    '500K–1M': 750000,
    '1M–5M': 3000000,
    '5M–25M': 15000000,
    '25M–50M': 37500000,
    '50M–100M': 75000000
}

capitol_trades_df['size_value'] = capitol_trades_df['size'].map(size_mapping)
capitol_trades_df['price'] = pd.to_numeric(capitol_trades_df['price'], errors='coerce')
capitol_trades_df['transaction_amount'] = capitol_trades_df['size_value'] * capitol_trades_df['price']

# Adjust gains based on transaction type
capitol_trades_df['gain'] = capitol_trades_df.apply(
    lambda x: x['transaction_amount'] if x['type'].lower() == 'sell' else -x['transaction_amount'], axis=1
)

# state and name of the politician
capitol_trades_df[['politician_name', 'politician_state']] = capitol_trades_df['politician'].str.extract(r'([^\d]+)([A-Z]{2})')

# Group by politician and state, and sum the total gains for each politician
politician_gains = capitol_trades_df.groupby(['politician_name', 'politician_state'])['gain'].sum().reset_index()

# politician with the highest gains in each state
top_politicians = politician_gains.loc[politician_gains.groupby('politician_state')['gain'].idxmax()]

top_politicians['text'] = top_politicians['politician_name'] + ": $" + top_politicians['gain'].round(2).astype(str)

# visualize the top-earning politician by state
fig = px.choropleth(
    top_politicians,
    locations='politician_state',
    locationmode='USA-states',
    color='gain',
    hover_name='politician_name',
    hover_data={'gain': True, 'text': True},
    scope='usa',
    color_continuous_scale='Viridis',
    labels={'gain': 'Total Gain ($)'},
    title='Top-Earning Politicians by State (Past 180 Days)'
)

fig.update_layout(
    geo=dict(
        lakecolor='rgb(255, 255, 255)',
        bgcolor='rgb(240, 240, 240)'
    )
)

fig.show()

# list of all politicians who traded in the past 180 days
unique_politicians = capitol_trades_df[['politician_name', 'politician_state']].drop_duplicates()

print(unique_politicians)

unique_politicians_df = pd.DataFrame(unique_politicians)
unique_politicians_df

                            politician_name politician_state
1                Lloyd DoggettDemocratHouse               TX
4                    Ro KhannaDemocratHouse               CA
616               John JamesRepublicanHouse               MI
743              Suzan DelBeneDemocratHouse               WA
752    Shelley Moore CapitoRepublicanSenate               WV
755   Marjorie Taylor GreeneRepublicanHouse               GA
769              Kathy ManningDemocratHouse               NC
837        Sheldon WhitehouseDemocratSenate               RI
847           Scott FranklinRepublicanHouse               FL
916            Jared MoskowitzDemocratHouse               FL
939           Thomas Kean JrRepublicanHouse               NJ
1031          Michael McCaulRepublicanHouse               TX
1125           Josh GottheimerDemocratHouse               NJ
1181          Susan CollinsRepublicanSenate               ME
1184             Greg LandsmanDemocratHouse               OH
1255              Rick A

,politician_name,politician_state
1,Lloyd DoggettDemocratHouse,TX
4,Ro KhannaDemocratHouse,CA
616,John JamesRepublicanHouse,MI
743,Suzan DelBeneDemocratHouse,WA
752,Shelley Moore CapitoRepublicanSenate,WV
755,Marjorie Taylor GreeneRepublicanHouse,GA
769,Kathy ManningDemocratHouse,NC
837,Sheldon WhitehouseDemocratSenate,RI
847,Scott FranklinRepublicanHouse,FL
916,Jared MoskowitzDemocratHouse,FL


In [13]:
# top politicians with their cumulative gains by state
top_politicians_summary = top_politicians[['politician_name', 'politician_state', 'gain']].copy()

# Renaming columns for better readability
top_politicians_summary.columns = ['Politician Name', 'State', 'Cumulative Gain ($)']

print(top_politicians_summary)

top_politicians_summary

                         Politician Name State  Cumulative Gain ($)
39      Tommy TubervilleRepublicanSenate    AL         9.876545e+07
8           John BoozmanRepublicanSenate    AR        -4.595250e+06
22             Nancy PelosiDemocratHouse    CA         8.233275e+08
38              Tom CarperDemocratSenate    DE         5.676000e+05
29         Scott FranklinRepublicanHouse    FL         4.806305e+07
25             Rick AllenRepublicanHouse    GA         1.215500e+07
10         Jonathan JacksonDemocratHouse    IL        -6.275100e+06
21          Morgan McGarveyDemocratHouse    KY         0.000000e+00
0              Bill KeatingDemocratHouse    MA        -1.530725e+06
34         Susan CollinsRepublicanSenate    ME         0.000000e+00
9              John JamesRepublicanHouse    MI         5.524860e+07
37              Tina SmithDemocratSenate    MN         1.115275e+07
4              Deborah RossDemocratHouse    NC         3.088800e+06
11          Josh GottheimerDemocratHouse    NJ  

,Politician Name,State,Cumulative Gain ($)
39,Tommy TubervilleRepublicanSenate,AL,9.876545e+07
8,John BoozmanRepublicanSenate,AR,-4.595250e+06
22,Nancy PelosiDemocratHouse,CA,8.233275e+08
38,Tom CarperDemocratSenate,DE,5.676000e+05
29,Scott FranklinRepublicanHouse,FL,4.806305e+07
25,Rick AllenRepublicanHouse,GA,1.215500e+07
10,Jonathan JacksonDemocratHouse,IL,-6.275100e+06
21,Morgan McGarveyDemocratHouse,KY,0.000000e+00
0,Bill KeatingDemocratHouse,MA,-1.530725e+06
34,Susan CollinsRepublicanSenate,ME,0.000000e+00
